In [5]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('../data/train_users_2.csv', parse_dates=[1,3])
df

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaT,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaT,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014-06-30,20140630235636,NaT,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF
213447,mhewnxesx9,2014-06-30,20140630235719,NaT,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF
213448,6o3arsjbb4,2014-06-30,20140630235754,NaT,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF
213449,jh95kwisub,2014-06-30,20140630235822,NaT,-unknown-,NaN,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF


## Datetime features:

In [ ]:
df[df.select_dtypes('datetime').columns]

,date_account_created,date_first_booking
0,2010-06-28,NaT
1,2011-05-25,NaT
2,2010-09-28,2010-08-02
3,2011-12-05,2012-09-08
4,2010-09-14,2010-02-18
...,...,...
213446,2014-06-30,NaT
213447,2014-06-30,NaT
213448,2014-06-30,NaT
213449,2014-06-30,NaT


In [ ]:
df['year_account_created'] = df.date_account_created.dt.year
df['month_account_created'] = df.date_account_created.dt.month
df['day_account_created'] = df.date_account_created.dt.day

df['year_first_booking'] = df.date_first_booking.dt.year
df['month_first_booking'] = df.date_first_booking.dt.month
df['day_first_booking'] = df.date_first_booking.dt.day

In [ ]:
# The missing values of date_first_booking are those who don't booked
df['year_first_booking'] = df.year_first_booking.fillna(-1)
df['month_first_booking'] = df.month_first_booking.fillna(-1)
df['day_first_booking'] = df.day_first_booking.fillna(-1)

## Numerical features:

In [ ]:
num_cols = df.select_dtypes('number').columns

df[num_cols].isna().sum()

timestamp_first_active        0
age                       87990
signup_flow                   0
year_account_created          0
month_account_created         0
day_account_created           0
year_first_booking            0
month_first_booking           0
day_first_booking             0
dtype: int64

In [ ]:
print('The percentage of nan values in age feature: ', round(df['age'].isna().mean() * 100, 2), '%')

The percentage of nan values in age feature:  41.22 %


### TimeStamp

In [ ]:
df.timestamp_first_active

0         20090319043255
1         20090523174809
2         20090609231247
3         20091031060129
4         20091208061105
               ...      
213446    20140630235636
213447    20140630235719
213448    20140630235754
213449    20140630235822
213450    20140630235824
Name: timestamp_first_active, Length: 213451, dtype: int64

In [ ]:
df['timestamp_first_active_date'] =  pd.to_datetime(df['timestamp_first_active'], format='%Y%m%d%H%M%S')

df['timestamp_first_active_date']

0        2009-03-19 04:32:55
1        2009-05-23 17:48:09
2        2009-06-09 23:12:47
3        2009-10-31 06:01:29
4        2009-12-08 06:11:05
                 ...        
213446   2014-06-30 23:56:36
213447   2014-06-30 23:57:19
213448   2014-06-30 23:57:54
213449   2014-06-30 23:58:22
213450   2014-06-30 23:58:24
Name: timestamp_first_active_date, Length: 213451, dtype: datetime64[ns]

In [ ]:
df['first_active_year'] = df.timestamp_first_active_date.dt.year
df['first_active_month'] = df.timestamp_first_active_date.dt.month
df['first_active_day'] = df.timestamp_first_active_date.dt.day

df['first_active_hour'] = df.timestamp_first_active_date.dt.hour
df['first_active_minute'] = df.timestamp_first_active_date.dt.minute

In [ ]:
# Feature selection (numerical features)
num_cols = df.select_dtypes('number').columns.drop(['timestamp_first_active', 'age', 'year_account_created','month_account_created','day_account_created','year_first_booking', 'month_first_booking', 'day_first_booking']).tolist()

### Correlation

In [ ]:
df[num_cols].isna().sum()

signup_flow            0
first_active_year      0
first_active_month     0
first_active_day       0
first_active_hour      0
first_active_minute    0
dtype: int64

In [ ]:
df[num_cols]

,signup_flow,first_active_year,first_active_month,first_active_day,first_active_hour,first_active_minute
0,0,2009,3,19,4,32
1,0,2009,5,23,17,48
2,3,2009,6,9,23,12
3,0,2009,10,31,6,1
4,0,2009,12,8,6,11
...,...,...,...,...,...,...
213446,0,2014,6,30,23,56
213447,0,2014,6,30,23,57
213448,0,2014,6,30,23,57
213449,25,2014,6,30,23,58


In [ ]:
df[num_cols].corr().style.background_gradient(cmap='coolwarm')

,signup_flow,first_active_year,first_active_month,first_active_day,first_active_hour,first_active_minute
signup_flow,1.000000,0.195165,-0.038870,0.014783,-0.019277,0.002848
first_active_year,0.195165,1.000000,-0.436526,0.003206,-0.012991,0.003181
first_active_month,-0.038870,-0.436526,1.000000,-0.013080,0.006555,-0.003174
first_active_day,0.014783,0.003206,-0.013080,1.000000,-0.001874,0.002510
first_active_hour,-0.019277,-0.012991,0.006555,-0.001874,1.000000,0.016368
first_active_minute,0.002848,0.003181,-0.003174,0.002510,0.016368,1.000000


## Object features:

In [ ]:
# Feature selection (categorical features)
cat_cols = df.select_dtypes('object').columns.drop('id').tolist()

cat_cols.remove('country_destination')

In [ ]:
df[cat_cols].isna().sum()

gender                        0
signup_method                 0
language                      0
affiliate_channel             0
affiliate_provider            0
first_affiliate_tracked    6065
signup_app                    0
first_device_type             0
first_browser                 0
dtype: int64

### Affiliate tracked

In [ ]:
df['first_affiliate_tracked'] = df.first_affiliate_tracked.fillna(df.first_affiliate_tracked.mode().iloc[0])

### Gender

In [ ]:
df.gender.value_counts(normalize=True)

-unknown-    0.448290
FEMALE       0.295342
MALE         0.255047
OTHER        0.001321
Name: gender, dtype: float64

In [ ]:
df.loc[df.gender.isin(['OTHER','-unknown-']), 'gender'] = np.nan

proba = df.gender.value_counts(normalize=True)
rep = np.random.choice(['FEMALE', 'MALE'], df.shape[0], p = proba)

df['gender'] = df.gender.fillna(pd.Series(rep))

### Encoder

In [ ]:
df[cat_cols]

,gender,signup_method,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,MALE,facebook,en,direct,direct,untracked,Web,Mac Desktop,Chrome
1,MALE,facebook,en,seo,google,untracked,Web,Mac Desktop,Chrome
2,FEMALE,basic,en,direct,direct,untracked,Web,Windows Desktop,IE
3,FEMALE,facebook,en,direct,direct,untracked,Web,Mac Desktop,Firefox
4,FEMALE,basic,en,direct,direct,untracked,Web,Mac Desktop,Chrome
...,...,...,...,...,...,...,...,...,...
213446,MALE,basic,en,sem-brand,google,omg,Web,Mac Desktop,Safari
213447,MALE,basic,en,direct,direct,linked,Web,Windows Desktop,Chrome
213448,MALE,basic,en,direct,direct,untracked,Web,Mac Desktop,Firefox
213449,FEMALE,basic,en,other,other,tracked-other,iOS,iPhone,Mobile Safari


In [ ]:
ordinal_encoder = OrdinalEncoder()

df[cat_cols] = ordinal_encoder.fit_transform(df[cat_cols])

In [ ]:
df[cat_cols]

,gender,signup_method,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,1.0,1.0,5.0,2.0,4.0,6.0,2.0,3.0,8.0
1,1.0,1.0,5.0,7.0,8.0,6.0,2.0,3.0,8.0
2,0.0,0.0,5.0,2.0,4.0,6.0,2.0,6.0,21.0
3,0.0,1.0,5.0,2.0,4.0,6.0,2.0,3.0,17.0
4,0.0,0.0,5.0,2.0,4.0,6.0,2.0,3.0,8.0
...,...,...,...,...,...,...,...,...,...
213446,1.0,0.0,5.0,5.0,8.0,3.0,2.0,3.0,41.0
213447,1.0,0.0,5.0,2.0,4.0,0.0,2.0,6.0,8.0
213448,1.0,0.0,5.0,2.0,4.0,6.0,2.0,3.0,17.0
213449,0.0,0.0,5.0,3.0,12.0,5.0,3.0,8.0,29.0


### Targer variable

In [ ]:
label_encoder = LabelEncoder()
df['country_destination'] = label_encoder.fit_transform(df.country_destination)

## Text features:

In [ ]:
sessions_df = pd.read_csv('../data/sessions.csv')
sessions_df

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0
...,...,...,...,...,...,...
10567732,9uqfg8txu3,dashboard,view,dashboard,Windows Desktop,556.0
10567733,9uqfg8txu3,edit,view,edit_profile,Windows Desktop,6624.0
10567734,9uqfg8txu3,webcam_upload,-unknown-,-unknown-,Windows Desktop,200125.0
10567735,9uqfg8txu3,active,-unknown-,-unknown-,-unknown-,17624.0


### Device type feature

In [ ]:
deviceType_df = sessions_df[['user_id', 'device_type']]

In [ ]:
deviceType_df

,user_id,device_type
0,d1mm9tcy42,Windows Desktop
1,d1mm9tcy42,Windows Desktop
2,d1mm9tcy42,Windows Desktop
3,d1mm9tcy42,Windows Desktop
4,d1mm9tcy42,Windows Desktop
...,...,...
10567732,9uqfg8txu3,Windows Desktop
10567733,9uqfg8txu3,Windows Desktop
10567734,9uqfg8txu3,Windows Desktop
10567735,9uqfg8txu3,-unknown-


In [ ]:
deviceType_df['new_device'] = deviceType_df.groupby('user_id')['device_type'].transform(lambda x: ' '.join(x.mode()))

deviceType_df = deviceType_df.drop_duplicates(subset='user_id')

<ipython-input-50-7afdb2241475>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deviceType_df['new_device'] = deviceType_df.groupby('user_id')['device_type'].transform(lambda x: ' '.join(x.mode()))


In [ ]:
deviceType_df[['user_id', 'new_device']]

,user_id,new_device
0,d1mm9tcy42,Windows Desktop
127,yo8nz8bqcq,Mac Desktop
136,4grx6yxeby,Windows Desktop
152,ncf87guaf0,Windows Desktop
304,4rvqpxoh3h,iPhone
...,...,...
10567515,cv0na2lf5a,Windows Desktop
10567608,zp8xfonng8,Android Phone
10567628,fa6260ziny,Windows Desktop
10567706,87k0fy4ugm,Mac Desktop


In [ ]:
label_encoder = LabelEncoder()
deviceType_df['new_device'] = label_encoder.fit_transform(deviceType_df.new_device)

<ipython-input-52-b4292b82330a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deviceType_df['new_device'] = label_encoder.fit_transform(deviceType_df.new_device)


In [ ]:
deviceType_df[['user_id', 'new_device']]

,user_id,new_device
0,d1mm9tcy42,38
127,yo8nz8bqcq,28
136,4grx6yxeby,38
152,ncf87guaf0,38
304,4rvqpxoh3h,44
...,...,...
10567515,cv0na2lf5a,38
10567608,zp8xfonng8,14
10567628,fa6260ziny,38
10567706,87k0fy4ugm,28


### Action features

In [ ]:
action_df = sessions_df[['user_id', 'action', 'action_type', 'action_detail']]
action_df

,user_id,action,action_type,action_detail
0,d1mm9tcy42,lookup,NaN,NaN
1,d1mm9tcy42,search_results,click,view_search_results
2,d1mm9tcy42,lookup,NaN,NaN
3,d1mm9tcy42,search_results,click,view_search_results
4,d1mm9tcy42,lookup,NaN,NaN
...,...,...,...,...
10567732,9uqfg8txu3,dashboard,view,dashboard
10567733,9uqfg8txu3,edit,view,edit_profile
10567734,9uqfg8txu3,webcam_upload,-unknown-,-unknown-
10567735,9uqfg8txu3,active,-unknown-,-unknown-


In [ ]:
action_df = action_df.fillna('')
action_df['actions'] = action_df['action'] + ' ' + action_df['action_type'] + ' ' + action_df['action_detail']

In [ ]:
action_df[['user_id', 'actions']]

,user_id,actions
0,d1mm9tcy42,lookup
1,d1mm9tcy42,search_results click view_search_results
2,d1mm9tcy42,lookup
3,d1mm9tcy42,search_results click view_search_results
4,d1mm9tcy42,lookup
...,...,...
10567732,9uqfg8txu3,dashboard view dashboard
10567733,9uqfg8txu3,edit view edit_profile
10567734,9uqfg8txu3,webcam_upload -unknown- -unknown-
10567735,9uqfg8txu3,active -unknown- -unknown-


In [ ]:
action_df['new_action'] = action_df.groupby('user_id')['actions'].transform(lambda x: ' '.join(x.astype(str).unique()))

action_df = action_df.drop_duplicates(subset='user_id')

In [ ]:
action_df[['user_id', 'new_action']]

,user_id,new_action
0,d1mm9tcy42,lookup search_results click view_search_resu...
127,yo8nz8bqcq,dashboard view dashboard create submit create_...
136,4grx6yxeby,verify -unknown- -unknown- create submit creat...
152,ncf87guaf0,lookup show view p3 search_results click vie...
304,4rvqpxoh3h,campaigns -unknown- -unknown- active -unknown-...
...,...,...
10567515,cv0na2lf5a,confirm_email click confirm_email_link authent...
10567608,zp8xfonng8,index view view_reservations campaigns colle...
10567628,fa6260ziny,index view view_search_results my view user_wi...
10567706,87k0fy4ugm,show update submit update_listing create -un...


In [ ]:
action_df['new_action'].fillna('')

0           lookup   search_results click view_search_resu...
127         dashboard view dashboard create submit create_...
136         verify -unknown- -unknown- create submit creat...
152         lookup   show view p3 search_results click vie...
304         campaigns -unknown- -unknown- active -unknown-...
                                  ...                        
10567515    confirm_email click confirm_email_link authent...
10567608    index view view_reservations campaigns   colle...
10567628    index view view_search_results my view user_wi...
10567706    show   update submit update_listing create -un...
10567721    show view user_profile update submit update_us...
Name: new_action, Length: 135484, dtype: object

In [ ]:
tfidVect = TfidfVectorizer(max_features = 100)
tfidVect.fit(action_df['new_action'].fillna(''))

X_tfid = tfidVect.transform(action_df['new_action'].fillna(''))

action_df_tfidVect = pd.DataFrame(X_tfid.toarray(), columns = tfidVect.get_feature_names_out())

In [ ]:
action_df_tfidVect

,account_notification_settings,active,agree_terms_check,ajax_check_dates,ajax_image_upload,ajax_lwlb_contact,ajax_refresh_subtotal,ask_question,at_checkpoint,authenticate,...,user_profile_content_update,user_reviews,user_social_connections,user_wishlists,verify,view,view_search_results,wishlist_content_update,your_listings,your_trips
0,0.000000,0.093806,0.0,0.0,0.0,0.0,0.090757,0.129888,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.105544,0.138440,0.080683,0.000000,0.00000
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.275164,0.0,0.0,0.000000,0.000000,0.331325,0.000000,0.168854,0.000000,0.00000
2,0.000000,0.092356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.137765,0.051956,0.000000,0.079436,0.000000,0.00000
3,0.000000,0.083156,0.0,0.0,0.0,0.0,0.080454,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.242333,0.000000,0.233903,0.122723,0.071523,0.000000,0.00000
4,0.000000,0.106746,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135479,0.118277,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.074601,...,0.083051,0.0,0.0,0.000000,0.088386,0.533339,0.000000,0.050964,0.113401,0.00000
135480,0.000000,0.179509,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.261562,0.000000,0.302956,0.000000,0.000000,0.000000,0.00000
135481,0.000000,0.000000,0.0,0.0,0.0,0.0,0.100105,0.000000,0.0,0.130268,...,0.000000,0.0,0.0,0.301525,0.000000,0.407451,0.152699,0.088993,0.000000,0.17193
135482,0.000000,0.000000,0.0,0.0,0.0,0.0,0.153635,0.000000,0.0,0.199927,...,0.000000,0.0,0.0,0.000000,0.000000,0.267998,0.117177,0.136581,0.000000,0.00000


### Action & secs elapsed features

In [ ]:
action_secs_df = sessions_df[['user_id', 'action', 'secs_elapsed']]

In [ ]:
action_secs_df

,user_id,action,secs_elapsed
0,d1mm9tcy42,lookup,319.0
1,d1mm9tcy42,search_results,67753.0
2,d1mm9tcy42,lookup,301.0
3,d1mm9tcy42,search_results,22141.0
4,d1mm9tcy42,lookup,435.0
...,...,...,...
10567732,9uqfg8txu3,dashboard,556.0
10567733,9uqfg8txu3,edit,6624.0
10567734,9uqfg8txu3,webcam_upload,200125.0
10567735,9uqfg8txu3,active,17624.0


In [ ]:
groupby_df = action_secs_df

groupby_df = groupby_df.groupby(['user_id', 'action']).agg({'secs_elapsed': 'mean'}).reset_index()
groupby_df = groupby_df.fillna(0)

In [ ]:
groupby_df

,user_id,action,secs_elapsed
0,00023iyk9l,ajax_refresh_subtotal,723.500000
1,00023iyk9l,callback,0.000000
2,00023iyk9l,confirm_email,567165.000000
3,00023iyk9l,dashboard,1115.500000
4,00023iyk9l,header_userpic,1599.000000
...,...,...,...
1732716,zzzlylp57e,personalize,5948.833333
1732717,zzzlylp57e,requested,4466.800000
1732718,zzzlylp57e,search_results,3752.700000
1732719,zzzlylp57e,show,2394.214286


In [ ]:
# Pivot the DataFrame to create the desired columns
df_pivot = groupby_df.pivot(index='user_id', columns='action', values='secs_elapsed').reset_index()

# Rename the columns to include the action names
df_pivot.columns = ['user_id'] + [f"{action}_average_in_secs" for action in groupby_df['action'].unique()]

df_pivot = df_pivot.fillna(0)

df_pivot

,user_id,ajax_refresh_subtotal_average_in_secs,callback_average_in_secs,confirm_email_average_in_secs,dashboard_average_in_secs,header_userpic_average_in_secs,index_average_in_secs,pending_average_in_secs,personalize_average_in_secs,requested_average_in_secs,...,acculynk_pin_pad_success_average_in_secs,deactivated_average_in_secs,special_offer_average_in_secs,deactivate_average_in_secs,events_average_in_secs,south-america_average_in_secs,reactivate_average_in_secs,update_message_average_in_secs,wishlists_average_in_secs,plaxo_cb_average_in_secs
0,00023iyk9l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0010k6l0om,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,001wyh0pz8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0028jgx1x1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,002qnbzfs5,33498.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135038,zzxox7jnrx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135039,zzy7t0y9cm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135040,zzysuoqg6x,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135041,zzywmcn0jv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
drop_list = []
for i in range(df_pivot.shape[1]):
  if sum(df_pivot[df_pivot.columns[i]] == 0) > df_pivot.shape[0] - df_pivot.shape[0]/12: # df_pivot.shape[0] - df_pivot.shape[0]/12 = 100 000
    drop_list.append(df_pivot.columns[i])

df_pivot = df_pivot.drop(drop_list, axis=1)

In [ ]:
df_pivot

,user_id,how_it_works_average_in_secs,update_average_in_secs,listings_average_in_secs,ajax_check_dates_average_in_secs,message_to_host_focus_average_in_secs,populate_help_dropdown_average_in_secs,reviews_new_average_in_secs,track_page_view_average_in_secs,terms_and_conditions_average_in_secs,...,new_session_average_in_secs,set_default_average_in_secs,city_count_average_in_secs,payoneer_signup_complete_average_in_secs,social-media_average_in_secs,badge_average_in_secs,acculynk_pin_pad_inactive_average_in_secs,sandy_average_in_secs,message_average_in_secs,use_mobile_site_average_in_secs
0,00023iyk9l,0.000000,0.0,0.0,723.500000,0.0,0.0,0.000000,0.00,567165.0,...,22079.00,6841.444444,0.0,110.000000,0.000,0.000000,733.0,0.000000,0.000000,0.0
1,0010k6l0om,0.000000,0.0,0.0,2513.750000,0.0,0.0,0.000000,0.00,0.0,...,5730.50,11474.300000,0.0,0.000000,93.375,0.000000,0.0,0.000000,0.000000,0.0
2,001wyh0pz8,5482.750000,0.0,0.0,0.000000,0.0,0.0,962.000000,8091.75,0.0,...,0.00,2379.833333,0.0,0.000000,0.000,775.000000,0.0,0.000000,50548.000000,0.0
3,0028jgx1x1,489.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,...,0.00,5874.400000,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0
4,002qnbzfs5,6399.551724,0.0,0.0,0.000000,0.0,0.0,48813.962963,0.00,14520.0,...,0.00,1837.193966,0.0,0.000000,0.000,1211.069444,0.0,2486.428571,3825.692308,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135038,zzxox7jnrx,0.000000,0.0,0.0,4542.800000,0.0,0.0,0.000000,0.00,43941.0,...,6636.75,1759.454545,597.5,0.000000,537.000,0.000000,14407.5,0.000000,98479.000000,14652.0
135039,zzy7t0y9cm,0.000000,0.0,0.0,664.000000,0.0,0.0,0.000000,0.00,25616.0,...,0.00,10.000000,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0
135040,zzysuoqg6x,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,...,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0
135041,zzywmcn0jv,0.000000,1061.0,280030.0,12618.636364,545.5,0.0,0.000000,0.00,0.0,...,25059.00,5195.437500,0.0,936.818182,0.000,0.000000,0.0,0.000000,0.000000,0.0


### Merge text features

In [ ]:
id = pd.DataFrame(sessions_df['user_id']).drop_duplicates(subset='user_id')

id = id.rename(columns={'user_id': 'id'})
id = id.reset_index(drop=True)

action_df_tfidVect = pd.concat([id, action_df_tfidVect], axis=1 )

In [ ]:
action_df_tfidVect

,id,account_notification_settings,active,agree_terms_check,ajax_check_dates,ajax_image_upload,ajax_lwlb_contact,ajax_refresh_subtotal,ask_question,at_checkpoint,...,user_profile_content_update,user_reviews,user_social_connections,user_wishlists,verify,view,view_search_results,wishlist_content_update,your_listings,your_trips
0,d1mm9tcy42,0.000000,0.093806,0.0,0.0,0.0,0.0,0.090757,0.129888,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.105544,0.138440,0.080683,0.000000,0.00000
1,yo8nz8bqcq,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.275164,0.0,0.0,0.000000,0.000000,0.331325,0.000000,0.168854,0.000000,0.00000
2,4grx6yxeby,0.000000,0.092356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.137765,0.051956,0.000000,0.079436,0.000000,0.00000
3,ncf87guaf0,0.000000,0.083156,0.0,0.0,0.0,0.0,0.080454,0.000000,0.0,...,0.000000,0.0,0.0,0.242333,0.000000,0.233903,0.122723,0.071523,0.000000,0.00000
4,4rvqpxoh3h,0.000000,0.106746,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135479,cv0na2lf5a,0.118277,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.083051,0.0,0.0,0.000000,0.088386,0.533339,0.000000,0.050964,0.113401,0.00000
135480,zp8xfonng8,0.000000,0.179509,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.261562,0.000000,0.302956,0.000000,0.000000,0.000000,0.00000
135481,fa6260ziny,0.000000,0.000000,0.0,0.0,0.0,0.0,0.100105,0.000000,0.0,...,0.000000,0.0,0.0,0.301525,0.000000,0.407451,0.152699,0.088993,0.000000,0.17193
135482,87k0fy4ugm,0.000000,0.000000,0.0,0.0,0.0,0.0,0.153635,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.267998,0.117177,0.136581,0.000000,0.00000


In [ ]:
deviceType_df[['user_id', 'new_device']]

,user_id,new_device
0,d1mm9tcy42,38
127,yo8nz8bqcq,28
136,4grx6yxeby,38
152,ncf87guaf0,38
304,4rvqpxoh3h,44
...,...,...
10567515,cv0na2lf5a,38
10567608,zp8xfonng8,14
10567628,fa6260ziny,38
10567706,87k0fy4ugm,28


In [ ]:
# merge actions and deviceType datasets:
action_device_df = pd.merge(action_df_tfidVect, deviceType_df[['user_id', 'new_device']], left_on="id", right_on="user_id")
action_device_df = action_device_df.drop('user_id', axis=1)
action_device_df

,id,account_notification_settings,active,agree_terms_check,ajax_check_dates,ajax_image_upload,ajax_lwlb_contact,ajax_refresh_subtotal,ask_question,at_checkpoint,...,user_reviews,user_social_connections,user_wishlists,verify,view,view_search_results,wishlist_content_update,your_listings,your_trips,new_device
0,d1mm9tcy42,0.000000,0.093806,0.0,0.0,0.0,0.0,0.090757,0.129888,0.0,...,0.0,0.0,0.000000,0.000000,0.105544,0.138440,0.080683,0.000000,0.00000,38
1,yo8nz8bqcq,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.331325,0.000000,0.168854,0.000000,0.00000,28
2,4grx6yxeby,0.000000,0.092356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.137765,0.051956,0.000000,0.079436,0.000000,0.00000,38
3,ncf87guaf0,0.000000,0.083156,0.0,0.0,0.0,0.0,0.080454,0.000000,0.0,...,0.0,0.0,0.242333,0.000000,0.233903,0.122723,0.071523,0.000000,0.00000,38
4,4rvqpxoh3h,0.000000,0.106746,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135479,cv0na2lf5a,0.118277,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.088386,0.533339,0.000000,0.050964,0.113401,0.00000,38
135480,zp8xfonng8,0.000000,0.179509,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.261562,0.000000,0.302956,0.000000,0.000000,0.000000,0.00000,14
135481,fa6260ziny,0.000000,0.000000,0.0,0.0,0.0,0.0,0.100105,0.000000,0.0,...,0.0,0.0,0.301525,0.000000,0.407451,0.152699,0.088993,0.000000,0.17193,38
135482,87k0fy4ugm,0.000000,0.000000,0.0,0.0,0.0,0.0,0.153635,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.267998,0.117177,0.136581,0.000000,0.00000,28


In [ ]:
df_pivot

,user_id,how_it_works_average_in_secs,update_average_in_secs,listings_average_in_secs,ajax_check_dates_average_in_secs,message_to_host_focus_average_in_secs,populate_help_dropdown_average_in_secs,reviews_new_average_in_secs,track_page_view_average_in_secs,terms_and_conditions_average_in_secs,...,new_session_average_in_secs,set_default_average_in_secs,city_count_average_in_secs,payoneer_signup_complete_average_in_secs,social-media_average_in_secs,badge_average_in_secs,acculynk_pin_pad_inactive_average_in_secs,sandy_average_in_secs,message_average_in_secs,use_mobile_site_average_in_secs
0,00023iyk9l,0.000000,0.0,0.0,723.500000,0.0,0.0,0.000000,0.00,567165.0,...,22079.00,6841.444444,0.0,110.000000,0.000,0.000000,733.0,0.000000,0.000000,0.0
1,0010k6l0om,0.000000,0.0,0.0,2513.750000,0.0,0.0,0.000000,0.00,0.0,...,5730.50,11474.300000,0.0,0.000000,93.375,0.000000,0.0,0.000000,0.000000,0.0
2,001wyh0pz8,5482.750000,0.0,0.0,0.000000,0.0,0.0,962.000000,8091.75,0.0,...,0.00,2379.833333,0.0,0.000000,0.000,775.000000,0.0,0.000000,50548.000000,0.0
3,0028jgx1x1,489.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,...,0.00,5874.400000,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0
4,002qnbzfs5,6399.551724,0.0,0.0,0.000000,0.0,0.0,48813.962963,0.00,14520.0,...,0.00,1837.193966,0.0,0.000000,0.000,1211.069444,0.0,2486.428571,3825.692308,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135038,zzxox7jnrx,0.000000,0.0,0.0,4542.800000,0.0,0.0,0.000000,0.00,43941.0,...,6636.75,1759.454545,597.5,0.000000,537.000,0.000000,14407.5,0.000000,98479.000000,14652.0
135039,zzy7t0y9cm,0.000000,0.0,0.0,664.000000,0.0,0.0,0.000000,0.00,25616.0,...,0.00,10.000000,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0
135040,zzysuoqg6x,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,...,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0
135041,zzywmcn0jv,0.000000,1061.0,280030.0,12618.636364,545.5,0.0,0.000000,0.00,0.0,...,25059.00,5195.437500,0.0,936.818182,0.000,0.000000,0.0,0.000000,0.000000,0.0


In [ ]:
# merge the 'average of an action' with 'actions & device' datasets
new_sessions_df = pd.merge(df_pivot, action_device_df, left_on = 'user_id', right_on = 'id')
new_sessions_df = new_sessions_df.drop('id', axis=1)
new_sessions_df

,user_id,how_it_works_average_in_secs,update_average_in_secs,listings_average_in_secs,ajax_check_dates_average_in_secs,message_to_host_focus_average_in_secs,populate_help_dropdown_average_in_secs,reviews_new_average_in_secs,track_page_view_average_in_secs,terms_and_conditions_average_in_secs,...,user_reviews,user_social_connections,user_wishlists,verify,view,view_search_results,wishlist_content_update,your_listings,your_trips,new_device
0,00023iyk9l,0.000000,0.0,0.0,723.500000,0.0,0.0,0.000000,0.00,567165.0,...,0.000000,0.000000,0.000000,0.000000,0.324277,0.170140,0.099158,0.0,0.191567,28
1,0010k6l0om,0.000000,0.0,0.0,2513.750000,0.0,0.0,0.000000,0.00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.236954,0.207207,0.120760,0.0,0.000000,28
2,001wyh0pz8,5482.750000,0.0,0.0,0.000000,0.0,0.0,962.000000,8091.75,0.0,...,0.000000,0.202788,0.222651,0.000000,0.257887,0.112756,0.000000,0.0,0.000000,11
3,0028jgx1x1,489.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.293886,0.192743,0.000000,0.0,0.000000,0
4,002qnbzfs5,6399.551724,0.0,0.0,0.000000,0.0,0.0,48813.962963,0.00,14520.0,...,0.092642,0.069418,0.000000,0.000000,0.147132,0.038598,0.000000,0.0,0.000000,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135038,zzxox7jnrx,0.000000,0.0,0.0,4542.800000,0.0,0.0,0.000000,0.00,43941.0,...,0.000000,0.000000,0.000000,0.093791,0.282977,0.092794,0.054081,0.0,0.104480,38
135039,zzy7t0y9cm,0.000000,0.0,0.0,664.000000,0.0,0.0,0.000000,0.00,25616.0,...,0.000000,0.000000,0.000000,0.000000,0.138536,0.000000,0.211808,0.0,0.000000,38
135040,zzysuoqg6x,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,38
135041,zzywmcn0jv,0.000000,1061.0,280030.0,12618.636364,545.5,0.0,0.000000,0.00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.152098,0.199505,0.116271,0.0,0.000000,38


In [ ]:
text_cols = new_sessions_df.select_dtypes('number').columns.tolist()

## Merge & Check data

In [ ]:
train_df = pd.merge(df, new_sessions_df, left_on='id', right_on='user_id')
target = pd.merge(df[['country_destination', 'id']], new_sessions_df['user_id'], left_on='id', right_on='user_id')

train_df = train_df.drop(['id', 'user_id'], axis=1)
target = target.drop(['id', 'user_id'], axis=1)
target = target.squeeze()

In [ ]:
sum(train_df[num_cols + cat_cols + text_cols].isna().sum())

0

In [ ]:
train_df[num_cols+cat_cols+text_cols]

,signup_flow,first_active_year,first_active_month,first_active_day,first_active_hour,first_active_minute,gender,signup_method,language,affiliate_channel,...,user_reviews,user_social_connections,user_wishlists,verify,view,view_search_results,wishlist_content_update,your_listings,your_trips,new_device
0,0,2014,1,1,0,9,1.0,0.0,5.0,6.0,...,0.0,0.000000,0.000000,0.000000,0.105544,0.138440,0.080683,0.000000,0.000000,38
1,0,2014,1,1,0,15,0.0,0.0,5.0,2.0,...,0.0,0.000000,0.000000,0.000000,0.331325,0.000000,0.168854,0.000000,0.000000,28
2,0,2014,1,1,0,16,1.0,0.0,5.0,5.0,...,0.0,0.000000,0.000000,0.137765,0.051956,0.000000,0.079436,0.000000,0.000000,38
3,0,2014,1,1,0,21,0.0,0.0,5.0,2.0,...,0.0,0.000000,0.242333,0.000000,0.233903,0.122723,0.071523,0.000000,0.000000,38
4,25,2014,1,1,0,26,1.0,0.0,5.0,2.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73401,0,2014,6,30,23,56,1.0,0.0,5.0,5.0,...,0.0,0.000000,0.000000,0.065949,0.323332,0.065248,0.038026,0.084613,0.073465,28
73402,0,2014,6,30,23,57,1.0,0.0,5.0,2.0,...,0.0,0.000000,0.225934,0.000000,0.392534,0.114418,0.066683,0.000000,0.000000,38
73403,0,2014,6,30,23,57,1.0,0.0,5.0,2.0,...,0.0,0.000000,0.000000,0.000000,0.289236,0.000000,0.147404,0.000000,0.000000,28
73404,25,2014,6,30,23,58,0.0,0.0,5.0,3.0,...,0.0,0.238933,0.000000,0.000000,0.202569,0.132853,0.000000,0.000000,0.000000,44


In [ ]:
#df[num_cols + cat_cols].to_csv('cleaned1_data.csv', index=False)
#!ls